# model

In [1]:
import torch
import torch.nn as nn
class DiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout_rate):
        super(DiRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size = embed_size, 
                    hidden_size = num_hiddens, 
                    num_layers = num_layers,
                    dropout = dropout_rate,
                    bidirectional = False)
        self.decoder = nn.Linear(num_hiddens, vocab_size) # 初始时间步和最终时间步的隐藏状态作为全连接层输入

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.embedding(inputs.permute(1, 0))
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 2 * 隐藏单元个数)。
#         print(outputs.shape)
        output = outputs.permute(1, 0, 2)
#         print(output.shape)
        outs = output.reshape(output.size(0)*output.size(1), output.size(2))
#         print(outs.shape)
        ret = self.decoder(outs)
        return ret

# data

In [2]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [3]:
class Corpus(object):
    def __init__(self, train_batch_size=20, eval_batch_size=10, bptt=35):
        self.bptt = bptt
        train_iter = WikiText2(split='train')
        self.tokenizer = get_tokenizer('basic_english')
        counter = Counter()
        for line in train_iter:
            counter.update(self.tokenizer(line))
        self.vocab = Vocab(counter)
        train_iter, val_iter, test_iter = WikiText2()
        train_data = self.data_process(train_iter)
        val_data = self.data_process(val_iter)
        test_data = self.data_process(test_iter)

        self.train_data = self.batchify(train_data, train_batch_size)
        self.val_data = self.batchify(val_data, eval_batch_size)
        self.test_data = self.batchify(test_data, eval_batch_size)

    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],
                           dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

    def batchify(self, data, batch_size):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Divide the dataset into batch_size parts.
        nbatch = data.size(0) // batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = data.narrow(0, 0, nbatch * batch_size)
        # Evenly divide the data across the batch_size batches.
        data = data.view(batch_size, -1).t().contiguous()
        return data.to(device)

    def get_batch(self, source, i):
        seq_len = min(self.bptt, len(source) - 1 - i)
        data = source[i:i+seq_len]
        target = source[i+1:i+1+seq_len].reshape(-1)
        return data, target

    def get_ntokens(self):
        return len(self.vocab.stoi)

# main

In [4]:
import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import data
import model
class args():
    emsize = 100
    nhid = 200
    nlayers = 5
    lr = 2
    epochs = 1000
    batch_size = 32
    bptt = 256
    dropout = 0.5
    tied = False
    seed = 1234
    cuda = True
    log_interval = 100
    save = 'model.pt'

In [5]:
data_loader = Corpus(train_batch_size=args.batch_size,
                     eval_batch_size=args.batch_size,
                     bptt=args.bptt)

In [6]:
train_data = data_loader.train_data
val_data = data_loader.val_data
test_data = data_loader.test_data

In [7]:
def get_batch(source, i, evaluation=False):
    seq_len = min(args.bptt, len(source) - 1 - i)
#     data = Variable(source[i:i+seq_len], volatile=evaluation)
#     target = Variable(source[i+1:i+1+seq_len].view(-1))
    data = source[i:i+seq_len].t()
    tmp = source[i+1:i+1+seq_len].t()
#     print(data.shape,tmp.shape)
    target = tmp.reshape(-1)
    return data, target

In [8]:
import torch
torch.__version__
print(train_data.size(0))
for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
    data, targets = get_batch(train_data, i)
    print(data.shape,targets.shape,targets[-1])
    if (batch>=2): break

64062
torch.Size([32, 256]) torch.Size([8192]) tensor(30, device='cuda:0')
torch.Size([32, 256]) torch.Size([8192]) tensor(7, device='cuda:0')
torch.Size([32, 256]) torch.Size([8192]) tensor(1348, device='cuda:0')


In [9]:
len(data_loader.vocab.itos)

28783

In [10]:
ntokens = len(data_loader.vocab.itos)
# model = RNNModel(args.model, ntokens, args.emsize, args.nhid,
#         args.nlayers, args.dropout, args.tied)
model = DiRNN(ntokens, args.emsize, args.nhid, args.nlayers, args.dropout)

In [11]:
if args.cuda:
    model = model.to(torch.device('cuda'))

In [17]:
model.eval()
model.train()
model

DiRNN(
  (embedding): Embedding(28783, 100)
  (encoder): LSTM(100, 200, num_layers=5, dropout=0.5)
  (decoder): Linear(in_features=200, out_features=28783, bias=True)
)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

In [14]:
def evaluate(data_source):
    model.eval()
    total_loss = 0
    total_words = 0
    for i in range(0, data_source.size(0) - 1, args.bptt):
        data, targets = get_batch(data_source, i)
        
        output = model(data)
        loss = criterion(output, targets)

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
    total_loss = float(total_loss.to(torch.device('cpu')))
#     print(total_loss,total_words)
    return total_loss / total_words, math.exp(total_loss / total_words)

def train():
    model.train()
    total_loss = 0.
    total_words = 0
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        model.zero_grad()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
        
#         if (batch % args.log_interval == 0 and batch > 0):
#             cur_loss = total_loss / args.log_interval
#             elapsed = time.time() - start_time
#             print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
#                     'loss {:5.2f} | ppl {:8.2f}'.format(
#                 233, batch, len(train_data) // args.bptt, args.lr,
#                 elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
#             total_loss = 0
#             start_time = time.time()

    cur_loss = total_loss / total_words
    print('train {:10d} words | loss {:5.5f} | ppl {:5.5}'.format(total_words, cur_loss, math.exp(cur_loss)))
    return cur_loss, math.exp(cur_loss)

In [15]:
import numpy as np
Losstrain = []
Lossval = []
Losstest = []

In [ ]:
import time
for T in range(args.epochs):
    print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
    trainl, trainp = train()
    vall, valp = evaluate(val_data)
    testl, testp = evaluate(test_data)
    
    Losstrain.append(trainp)
    Lossval.append(valp)
    Losstest.append(testp)
    np.save('model-1-train',np.array(Losstrain))
    np.save('model-1-val',np.array(Lossval))
    np.save('model-1-test',np.array(Losstest))
    
    print('  valid {:5.5f} | test {:5.5f}'.format(valp, testp))
    print('-'*80)

Round :  0    2021-05-18 01:25:06
train    2049952 words | loss 7.58704 | ppl 1972.5
  valid 1615.58013 | test 1495.74959
--------------------------------------------------------------------------------
Round :  1    2021-05-18 01:26:12
train    2049952 words | loss 7.15098 | ppl 1275.4
  valid 927.75286 | test 878.28103
--------------------------------------------------------------------------------
Round :  2    2021-05-18 01:27:18
train    2049952 words | loss 7.07087 | ppl 1177.2
  valid 1122.52124 | test 1055.26744
--------------------------------------------------------------------------------
Round :  3    2021-05-18 01:28:23
train    2049952 words | loss 7.02944 | ppl 1129.4
  valid 924.43286 | test 875.71340
--------------------------------------------------------------------------------
Round :  4    2021-05-18 01:29:29
train    2049952 words | loss 7.00441 | ppl 1101.5
  valid 897.92834 | test 849.49387
------------------------------------------------------------------------

train    2049952 words | loss 6.02665 | ppl 414.33
  valid 340.24587 | test 316.96882
--------------------------------------------------------------------------------
Round :  42    2021-05-18 02:11:22
train    2049952 words | loss 6.00962 | ppl 407.33
  valid 350.40937 | test 326.75338
--------------------------------------------------------------------------------
Round :  43    2021-05-18 02:12:28
train    2049952 words | loss 5.99666 | ppl 402.08
  valid 346.18363 | test 322.65625
--------------------------------------------------------------------------------
Round :  44    2021-05-18 02:13:34
train    2049952 words | loss 5.98033 | ppl 395.57
  valid 330.15818 | test 307.39446
--------------------------------------------------------------------------------
Round :  45    2021-05-18 02:14:40
train    2049952 words | loss 5.96487 | ppl 389.5
  valid 329.68397 | test 306.52734
--------------------------------------------------------------------------------
Round :  46    2021-05-18 

train    2049952 words | loss 5.53714 | ppl 253.95
  valid 243.11044 | test 225.11802
--------------------------------------------------------------------------------
Round :  83    2021-05-18 02:56:38
train    2049952 words | loss 5.52822 | ppl 251.7
  valid 238.58446 | test 220.55432
--------------------------------------------------------------------------------
Round :  84    2021-05-18 02:57:44
train    2049952 words | loss 5.51891 | ppl 249.36
  valid 239.22095 | test 221.46184
--------------------------------------------------------------------------------
Round :  85    2021-05-18 02:58:51
train    2049952 words | loss 5.51224 | ppl 247.71
  valid 235.82150 | test 217.96191
--------------------------------------------------------------------------------
Round :  86    2021-05-18 02:59:57
train    2049952 words | loss 5.50267 | ppl 245.35
  valid 239.41018 | test 221.11925
--------------------------------------------------------------------------------
Round :  87    2021-05-18 

train    2049952 words | loss 5.26820 | ppl 194.07
  valid 208.75834 | test 193.89561
--------------------------------------------------------------------------------
Round :  124    2021-05-18 03:41:57
train    2049952 words | loss 5.26692 | ppl 193.82
  valid 209.18922 | test 194.02595
--------------------------------------------------------------------------------
Round :  125    2021-05-18 03:43:03
train    2049952 words | loss 5.25896 | ppl 192.28
  valid 215.19247 | test 199.51053
--------------------------------------------------------------------------------
Round :  126    2021-05-18 03:44:09
train    2049952 words | loss 5.25709 | ppl 191.92
  valid 208.42215 | test 193.75364
--------------------------------------------------------------------------------
Round :  127    2021-05-18 03:45:15
train    2049952 words | loss 5.37059 | ppl 214.99
  valid 223.65740 | test 206.47048
--------------------------------------------------------------------------------
Round :  128    2021-

train    2049952 words | loss 5.35513 | ppl 211.69
  valid 218.44861 | test 204.16822
--------------------------------------------------------------------------------
Round :  165    2021-05-18 04:27:11
train    2049952 words | loss 5.53473 | ppl 253.34
  valid 269.57758 | test 250.48531
--------------------------------------------------------------------------------
Round :  166    2021-05-18 04:28:17
train    2049952 words | loss 5.54529 | ppl 256.03
  valid 233.03387 | test 217.47472
--------------------------------------------------------------------------------
Round :  167    2021-05-18 04:29:23
train    2049952 words | loss 5.47027 | ppl 237.52
  valid 234.34448 | test 218.20190
--------------------------------------------------------------------------------
Round :  168    2021-05-18 04:30:30
train    2049952 words | loss 5.43241 | ppl 228.7
  valid 228.71549 | test 212.51392
--------------------------------------------------------------------------------
Round :  169    2021-0

...

train    2049952 words | loss 4.45541 | ppl 86.092
  valid 176.40908 | test 165.94160
--------------------------------------------------------------------------------
Round :  700    2021-05-18 14:20:38
train    2049952 words | loss 4.45456 | ppl 86.018
  valid 178.80000 | test 168.22083
--------------------------------------------------------------------------------
Round :  701    2021-05-18 14:21:45
train    2049952 words | loss 4.45227 | ppl 85.821
  valid 177.22629 | test 166.64999
--------------------------------------------------------------------------------
Round :  702    2021-05-18 14:22:51
train    2049952 words | loss 4.44950 | ppl 85.584
  valid 178.56203 | test 167.74552
--------------------------------------------------------------------------------
Round :  703    2021-05-18 14:23:57
train    2049952 words | loss 4.45188 | ppl 85.788
  valid 177.22267 | test 166.60133
--------------------------------------------------------------------------------
Round :  704    2021-

train    2049952 words | loss 4.43190 | ppl 84.091
  valid 178.44025 | test 167.59480
--------------------------------------------------------------------------------
Round :  741    2021-05-18 15:05:54
train    2049952 words | loss 4.42979 | ppl 83.914
  valid 177.27838 | test 166.07847
--------------------------------------------------------------------------------
Round :  742    2021-05-18 15:07:00
train    2049952 words | loss 4.42774 | ppl 83.742
  valid 177.23579 | test 165.35405
--------------------------------------------------------------------------------
Round :  743    2021-05-18 15:08:07
train    2049952 words | loss 4.42544 | ppl 83.549
  valid 180.12808 | test 168.51159
--------------------------------------------------------------------------------
Round :  744    2021-05-18 15:09:13
train    2049952 words | loss 4.42349 | ppl 83.387
  valid 177.75868 | test 166.69203
--------------------------------------------------------------------------------
Round :  745    2021-

train    2049952 words | loss 4.40392 | ppl 81.77
  valid 179.26728 | test 167.35007
--------------------------------------------------------------------------------
Round :  782    2021-05-18 15:51:12
train    2049952 words | loss 4.40105 | ppl 81.537
  valid 178.44951 | test 167.15728
--------------------------------------------------------------------------------
Round :  783    2021-05-18 15:52:18
train    2049952 words | loss 4.39857 | ppl 81.334
  valid 176.42575 | test 165.47401
--------------------------------------------------------------------------------
Round :  784    2021-05-18 15:53:25
train    2049952 words | loss 4.39975 | ppl 81.431
  valid 178.01852 | test 166.57274
--------------------------------------------------------------------------------
Round :  785    2021-05-18 15:54:31
train    2049952 words | loss 4.40073 | ppl 81.51
  valid 182.31132 | test 170.69001
--------------------------------------------------------------------------------
Round :  786    2021-05

train    2049952 words | loss 4.37743 | ppl 79.633
  valid 181.82094 | test 170.33085
--------------------------------------------------------------------------------
Round :  823    2021-05-18 16:37:26
train    2049952 words | loss 4.37749 | ppl 79.638
  valid 180.13196 | test 168.90716
--------------------------------------------------------------------------------
Round :  824    2021-05-18 16:38:33
train    2049952 words | loss 4.37518 | ppl 79.454
  valid 176.07817 | test 165.34559
--------------------------------------------------------------------------------
Round :  825    2021-05-18 16:39:40
train    2049952 words | loss 4.37684 | ppl 79.586
  valid 178.43411 | test 167.86261
--------------------------------------------------------------------------------
Round :  826    2021-05-18 16:40:48
train    2049952 words | loss 4.37638 | ppl 79.55
  valid 178.08413 | test 167.02401
--------------------------------------------------------------------------------
Round :  827    2021-0

train    2049952 words | loss 4.35511 | ppl 77.875
  valid 178.72256 | test 167.49217
--------------------------------------------------------------------------------
Round :  864    2021-05-18 17:52:50
train    2049952 words | loss 4.35286 | ppl  77.7
  valid 179.12611 | test 168.24257
--------------------------------------------------------------------------------
Round :  865    2021-05-18 17:54:53
train    2049952 words | loss 4.35269 | ppl 77.687
  valid 178.50707 | test 167.61880
--------------------------------------------------------------------------------
Round :  866    2021-05-18 17:56:56
train    2049952 words | loss 4.35258 | ppl 77.679
  valid 177.70427 | test 167.15814
--------------------------------------------------------------------------------
Round :  867    2021-05-18 17:59:00
train    2049952 words | loss 4.35074 | ppl 77.536
  valid 178.31086 | test 167.66819
--------------------------------------------------------------------------------
Round :  868    2021-0

In [ ]:
# print('  valid ',evaluate(val_data))
# print('  test  ',evaluate(test_data))

In [ ]:
# import time
# for T in range(100):
#     print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
#     print('-'*80)

In [ ]:
# if 1:
#     x = torch.tensor([[1000,1],[100,1],[1000,1]],dtype=float)
#     y = torch.tensor([0,1,0])
#     print(criterion(x,y))

In [ ]:
# -math.log(0.1)